# Drug Name Entity Classifier
## AHLT - MIRI 2018



## Initialization

Load needed modules and specify the working directory

In [1]:
# Load needed packages
import pandas as pd
import numpy as np
import scipy


import time # Execution time of some blocks
from nltk.tag import StanfordPOSTagger

# sklearn package
from sklearn.metrics import make_scorer
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics


# Import our defined functions
from NER_functions import *
from datasetBuilder import *
from feature_creation_ner import *

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# init time
first_init = time.time()

In [3]:
train_dirs_whereto_parse = ['data/Train/DrugBank','data/Train/MedLine']
test_dirs_whereto_parse = ['data/Test/Test for DrugNER task/DrugBank','data/Test/Test for DrugNER task/MedLine']

## Reading the train and test data from the XML files
Accessing to all the files of the directory and storing id's and text's in two arrays.
We have also added the token 'STOP' at the end of each sentence.

In [4]:
train_texts_entities = createSet(train_dirs_whereto_parse)
test_texts_entities = createSet(test_dirs_whereto_parse)

In [5]:
print(train_texts_entities[3])

('In controlled clinical trials of AUGMENTIN XR, 22 patients received concomitant allopurinol and AUGMENTIN XR.', [('AUGMENTIN XR', 'brand'), ('allopurinol', 'drug'), ('AUGMENTIN XR', 'brand')])


Before computing features, I want the input data to have a special format:

In [6]:
# we want each dataset with the following format: 
# for ['have','Ibuprofeno'] ---- [('hola','V','O'),('Ibuprofeno','NN','B')]

def buildSet(text_entities):
    dataset = []
    a = 0
    for text,drugs in text_entities:
        a+=1
        print(a)
        print('drugs: ',drugs)
        # tokenizing
        tokenized_sentence = nltk.word_tokenize(text)
        # BIO tagging
        tokens_tags = BIOTagger(text, drugs)
        # POS tagging
        tokens_pos = nltk.pos_tag(tokenized_sentence)
        text_triples = []
        for idx,token in enumerate(tokenized_sentence):
            text_triples.append((token,tokens_pos[idx][1],tokens_tags[idx][1]))
        dataset.append(text_triples)
    
    return dataset

# train_set and test_set are list of list of triples; each list of triples refers to a different sentence
train_set = buildSet(train_texts_entities) 
test_set = buildSet(test_texts_entities)

1
drugs:  [('calcium', 'drug'), ('EMCYT', 'brand')]
2
drugs:  [('allopurinol', 'drug'), ('ampicillin', 'drug'), ('ampicillin', 'drug')]
3
drugs:  [('ampicillin', 'drug'), ('allopurinol', 'drug')]
4
drugs:  [('AUGMENTIN XR', 'brand'), ('allopurinol', 'drug'), ('AUGMENTIN XR', 'brand')]
5
drugs:  [('AUGMENTIN XR', 'brand'), ('allopurinol', 'drug')]
6
drugs:  [('broad-spectrum antibiotics', 'group'), ('AUGMENTIN XR', 'brand'), ('contraceptives', 'group')]
7
drugs:  [('PROCRIT', 'brand')]
8
drugs:  [('central nervous system depressants', 'group'), ('benzodiazepines', 'group')]
9
drugs:  [('ROMAZICON', 'brand'), ('narcotics', 'group'), ('anesthetics', 'group'), ('muscle relaxants', 'group')]
10
drugs:  [('ROMAZICON', 'brand'), ('cyclic antidepressants', 'group'), ('benzodiazepine', 'group'), ('flumazenil', 'drug')]
11
drugs:  [('ROMAZICON', 'brand')]
12
drugs:  [('ROMAZICON', 'brand'), ('benzodiazepine', 'group')]
13
drugs:  [('ROMAZICON', 'brand'), ('benzodiazepines', 'group')]
14
drugs:  

120
drugs:  [('bupropion', 'drug')]
121
drugs:  [('bupropion', 'drug')]
122
drugs:  [('bupropion', 'drug'), ('antidepressants', 'group'), ('nortriptyline', 'drug'), ('imipramine', 'drug'), ('desipramine', 'drug'), ('paroxetine', 'drug'), ('fluoxetine', 'drug'), ('sertraline', 'drug'), ('antipsychotics', 'group'), ('haloperidol', 'drug'), ('risperidone', 'drug'), ('thioridazine', 'drug'), ('beta-blockers', 'group'), ('metoprolol', 'drug'), ('Type 1C antiarrhythmics', 'group'), ('propafenone', 'drug'), ('flecainide', 'drug')]
123
drugs:  [('bupropion', 'drug')]
124
drugs:  [('MAO Inhibitors', 'group'), ('bupropion', 'drug'), ('MAO inhibitor', 'group'), ('phenelzine', 'drug')]
125
drugs:  [('Levodopa', 'drug'), ('Amantadine', 'drug'), ('bupropion', 'drug'), ('levodopa', 'drug'), ('amantadine', 'drug')]
126
drugs:  [('WELLBUTRIN', 'brand'), ('levodopa', 'drug'), ('amantadine', 'drug')]
127
drugs:  [('WELLBUTRIN', 'brand'), ('antipsychotics', 'group'), ('antidepressants', 'group'), ('theoph

214
drugs:  [('Phenobarbital', 'drug'), ('Primidone', 'drug'), ('tiagabine', 'drug'), ('phenobarbital', 'drug'), ('primidone', 'drug')]
215
drugs:  [('tiagabine', 'drug'), ('phenobarbital', 'drug'), ('primidone', 'drug')]
216
drugs:  [('AEDs', 'group'), ('GABITRIL', 'brand'), ('Carbamazepine', 'drug'), ('tiagabine', 'drug'), ('carbamazepine', 'drug'), ('AEDs', 'group')]
217
drugs:  [('Phenytoin', 'drug'), ('tiagabine', 'drug'), ('phenytoin', 'drug'), ('AEDs', 'group')]
218
drugs:  [('Phenobarbital', 'drug'), ('Primidone', 'drug'), ('tiagabine', 'drug'), ('phenobarbital', 'drug'), ('primidone', 'drug'), ('AEDs', 'group')]
219
drugs:  [('Valproate', 'drug'), ('tiagabine', 'drug'), ('valproate', 'drug'), ('tiagabine', 'drug'), ('valproate', 'drug'), ('tiagabine', 'drug')]
220
drugs:  [('GABITRIL', 'brand'), ('Cimetidine', 'drug'), ('cimetidine', 'drug'), ('tiagabine', 'drug'), ('tiagabine', 'drug')]
221
drugs:  [('Theophylline', 'drug'), ('tiagabine', 'drug'), ('theophylline', 'drug')]
22

313
drugs:  [('antidiabetic drug', 'group')]
314
drugs:  [('reserpine', 'drug')]
315
drugs:  [('beta-blockers', 'group')]
316
drugs:  [('epinephrine', 'drug')]
317
drugs:  [('theophylline', 'drug'), ('quinolone', 'group')]
318
drugs:  [('theophylline', 'drug'), ('quinolones', 'group'), ('theophylline', 'drug')]
319
drugs:  [('theophylline', 'drug'), ('theophylline', 'drug')]
320
drugs:  [('Quinolones', 'group'), ('caffeine', 'drug')]
321
drugs:  [('caffeine', 'drug')]
322
drugs:  [('Quinolones', 'group'), ('nalidixic acid', 'drug'), ('anticoagulant', 'group'), ('warfarin', 'drug')]
323
drugs:  [('Nitrofurantoin', 'drug'), ('nalidixic acid', 'drug')]
324
drugs:  [('Antacids', 'group'), ('magnesium', 'drug'), ('aluminum', 'drug'), ('calcium', 'drug')]
325
drugs:  [('sucralfate', 'drug'), ('iron', 'drug')]
326
drugs:  [('multivitamins', 'group'), ('zinc', 'drug')]
327
drugs:  [('Videx', 'brand'), ('Didanosine', 'drug'), ('quinolones', 'group')]
328
drugs:  [('nalidixic acid', 'drug')]
329

453
drugs:  [('Vaccines', 'group'), ('corticosteroid', 'group'), ('live vaccines', 'group'), ('inactivated vaccines', 'group')]
454
drugs:  [('Corticosteroids', 'group'), ('live attenuated vaccines', 'group')]
455
drugs:  [('vaccines', 'group'), ('corticosteroid', 'group')]
456
drugs:  [('Cytosine arabinoside', 'drug'), ('cytostatic agent', 'group'), ('flucytosine', 'drug')]
457
drugs:  [('flucytosine', 'drug')]
458
drugs:  [('Ancobon', 'brand')]
459
drugs:  [('HYDROXYZINE', 'drug'), ('CENTRAL NERVOUS SYSTEM DEPRESSANTS', 'group'), ('NARCOTICS', 'group'), ('NON-NARCOTIC ANALGESICS', 'group'), ('BARBITURATES', 'group')]
460
drugs:  [('central nervous system depressants', 'group'), ('hydroxyzine', 'drug')]
461
drugs:  [('Atarax', 'brand')]
462
drugs:  [('CNS depressant drugs', 'group'), ('alcohol', 'drug')]
463
drugs:  [('ZOLADEX', 'brand')]
464
drugs:  [('Amantadine', 'drug'), ('tricyclic antidepressants', 'group'), ('MAOIs', 'group'), ('clidinium', 'drug')]
465
drugs:  [('Clidinium', '

...

4940
drugs:  [('digoxin', 'drug'), ('digoxin', 'drug'), ('amiodarone', 'drug')]
4941
drugs:  [('amiodarone', 'drug'), ('digoxin', 'drug')]
4942
drugs:  [('Digoxin', 'drug')]
4943
drugs:  [('Digoxin', 'drug')]
4944
drugs:  [('Amiodarone', 'drug'), ('digoxin', 'drug')]
4945
drugs:  [('digoxin', 'drug'), ('amiodarone', 'drug'), ('digoxin', 'drug')]
4946
drugs:  [('alcohol', 'drug')]
4947
drugs:  [('alcohol', 'drug'), ('alcohol', 'drug')]
4948
drugs:  [('ethanol', 'drug'), ('physostigmine', 'drug'), ('atropine', 'drug')]
4949
drugs:  [('Ethanol', 'drug')]
4950
drugs:  [('Physostigmine', 'drug')]
4951
drugs:  [('atropine', 'drug')]
4952
drugs:  [('Physostigmine', 'drug'), ('alcohol', 'drug'), ('alcohol', 'drug')]
4953
drugs:  [('atropine', 'drug')]
4954
drugs:  [('Atropine', 'drug'), ('alcohol', 'drug')]
4955
drugs:  [('ethanol', 'drug')]
4956
drugs:  [('cerulein', 'drug_n'), ('3H-spiroperidol', 'drug_n'), ('neuroleptics', 'group'), ('haloperidol', 'drug'), ('pyreneperone', 'drug_n'), ('3H-

drugs:  [('18-Methoxycoronaridine', 'drug_n'), ('18-MC', 'drug_n'), ('ibogaine', 'drug_n')]
5143
drugs:  [('18-MC', 'drug_n')]
5144
drugs:  [('ibogaine', 'drug_n'), ('18-MC', 'drug_n'), ('morphine', 'drug'), ('cocaine', 'drug'), ('ethanol', 'drug'), ('nicotine', 'drug')]
5145
drugs:  [('ibogaine', 'drug_n'), ('18-MC', 'drug_n')]
5146
drugs:  [('ibogaine', 'drug_n'), ('18-MC', 'drug_n'), ('opioid', 'group')]
5147
drugs:  [('ibogaine', 'drug_n'), ('18-MC', 'drug_n'), ('ibogaine', 'drug_n')]
5148
drugs:  [('ibogaine', 'drug_n'), ('18-MC', 'drug_n'), ('morphine', 'drug'), ('nicotine', 'drug')]
5149
drugs:  [('ibogaine', 'drug_n'), ('cocaine', 'drug')]
5150
drugs:  [('ibogaine', 'drug_n'), ('18-MC', 'drug_n')]
5151
drugs:  [('Ibogaine', 'drug_n'), ('18-MC', 'drug_n'), ('morphine', 'drug')]
5152
drugs:  [('morphine', 'drug'), ('morphine', 'drug')]
5153
drugs:  [('Ibogaine', 'drug_n')]
5154
drugs:  [('18-MC', 'drug_n')]
5155
drugs:  [('Ibogaine', 'drug_n'), ('18-MC', 'drug_n')]
5156
drugs:  [

5370
drugs:  [('phenobarbital', 'drug')]
5371
drugs:  [('KRM-1648', 'drug_n'), ('ofloxacin', 'drug')]
5372
drugs:  [('benzoxazinorifamycin', 'drug_n'), ('KRM-1648', 'drug_n'), ('ofloxacin', 'drug')]
5373
drugs:  [('KRM-1648', 'drug_n')]
5374
drugs:  [('rifampicin', 'drug'), ('rifabutin', 'drug')]
5375
drugs:  [('ofloxacin', 'drug'), ('KRM-1648', 'drug_n'), ('rifampicin', 'drug'), ('rifabutin', 'drug'), ('ofloxacin', 'drug')]
5376
drugs:  [('KRM-1648', 'drug_n')]
5377
drugs:  [('budesonide', 'drug'), ('Survanta', 'brand'), ('lung surfactant', 'group')]
5378
drugs:  [('pulmonary surfactant', 'group'), ('Survanta', 'brand'), ('budesonide', 'drug')]
5379
drugs:  [('budesonide', 'drug'), ('ethanol', 'drug')]
5380
drugs:  [('solvent', 'group')]
5381
drugs:  [('Survanta', 'brand'), ('budesonide', 'drug')]
5382
drugs:  [('Survanta', 'brand'), ('sodium dodecyl sulfate', 'drug_n')]
5383
drugs:  [('budesonide', 'drug'), ('budesonide', 'drug'), ('Survanta', 'brand')]
5384
drugs:  [('lung surfactan

29
drugs:  [('Pegaptanib', 'drug')]
30
drugs:  [('Macugen', 'brand'), ('pegaptanib', 'drug')]
31
drugs:  [('anticholinergic agents', 'group')]
32
drugs:  [('Alcohol', 'drug'), ('primaquine', 'drug')]
33
drugs:  [('methyl aminolevulinate', 'drug'), ('anesthetics', 'group')]
34
drugs:  [('methyl aminolevulinate', 'drug')]
35
drugs:  [('cimetidine', 'drug'), ('digoxin', 'drug'), ('sevelamer', 'drug'), ('theophylline', 'drug')]
36
drugs:  [('Telbivudine', 'drug'), ('telbivudine', 'drug')]
37
drugs:  [('telbivudine', 'drug'), ('telbivudine', 'drug'), ('telbivudine', 'drug')]
38
drugs:  [('Geref', 'brand'), ('Geref', 'brand')]
39
drugs:  [('BEXXAR', 'brand')]
40
drugs:  [('murine monoclonal antibodies', 'group')]
41
drugs:  [('Palonosetron', 'drug')]
42
drugs:  [('palonosetron', 'drug')]
43
drugs:  [('palonosetron', 'drug')]
44
drugs:  [('palonosetron', 'drug'), ('metoclopramide', 'drug')]
45
drugs:  [('ALOXI', 'brand'), ('corticosteroids', 'group'), ('analgesics', 'group'), ('antiemetics', 

234
drugs:  [('propiverine hydrochloride', 'drug')]
235
drugs:  [('propiverine hydrochloride', 'drug'), ('cetirizine di-hydrochloride', 'drug')]
236
drugs:  [('ethyl acetate', 'drug_n'), ('methanol-ammonium acetate', 'drug_n')]
237
drugs:  [('propiverine', 'drug')]
238
drugs:  [('propiverine hydrochloride', 'drug')]
239
drugs:  [('propiverine hydrochloride', 'drug')]
240
drugs:  [('antituberculosis drugs', 'group'), ('antiretroviral drugs', 'group')]
241
drugs:  [('antiretroviral', 'group'), ('antituberculosis', 'group')]
242
drugs:  [('Nitric oxide', 'drug')]
243
drugs:  [('opiate', 'group')]
244
drugs:  [('opiate', 'group')]
245
drugs:  [('opiate', 'group')]
246
drugs:  [('opiate', 'group')]
247
drugs:  [('opiate', 'group'), ('opiate', 'group')]
248
drugs:  [('opiate', 'group')]
249
drugs:  [('opiate', 'group'), ('morphine', 'drug')]
250
drugs:  [('opiate', 'group'), ('opiate', 'group'), ('opiate', 'group')]
251
drugs:  [('Ticagrelor', 'drug'), ('antiplatelet agent', 'group')]
252
dr

In [7]:
# computing the Distribution of frequencies that we will need later for a feature

list_of_tokens = [tuples[0] for row in train_set for tuples in row]
freqDistribution = nltk.FreqDist(list_of_tokens)

In [8]:
frequencies = [v for k,v in freqDistribution.items()]
frequencies = list(set(sorted(frequencies)))
max_pos = round(len(frequencies)*20/100) # we only take the 20 small frequencies
more_rare_freq = frequencies[0:max_pos]

## FEATURES 

In [9]:
with(open('data/DrugBank_names_DB.txt', 'r')) as f:
    drugbank_db = f.read().splitlines()

# remember s is a triple defined as follows: (token, POS tag, BIO tag)
X_train = [sent2features(s,drugbank_db,freqDistribution,more_rare_freq) for s in train_set]
y_train = [sent2labels(s) for s in train_set]

X_test = [sent2features(s,drugbank_db,freqDistribution,more_rare_freq) for s in test_set]
y_test = [sent2labels(s) for s in test_set]


In [10]:
X_train[598][5]

{'+1:bigrams': ['ma', 'ay'],
 '+1:postag': 'MD',
 '+1:postag[:2]': 'MD',
 '+1:trigrams': ['may'],
 '+1:unigrams': ['m', 'a', 'y'],
 '+2:bigrams': ['le', 'ea', 'ad'],
 '+2:postag': 'VB',
 '+2:postag[:2]': 'VB',
 '+2:trigrams': ['lea', 'ead'],
 '+2:unigrams': ['l', 'e', 'a', 'd'],
 '-1:bigrams': ['ga',
  'as',
  'st',
  'tr',
  'ro',
  'oi',
  'in',
  'nt',
  'te',
  'es',
  'st',
  'ti',
  'in',
  'na',
  'al'],
 '-1:postag': 'JJ',
 '-1:postag[:2]': 'JJ',
 '-1:trigrams': ['gas',
  'ast',
  'str',
  'tro',
  'roi',
  'oin',
  'int',
  'nte',
  'tes',
  'est',
  'sti',
  'tin',
  'ina',
  'nal'],
 '-1:unigrams': ['g',
  'a',
  's',
  't',
  'r',
  'o',
  'i',
  'n',
  't',
  'e',
  's',
  't',
  'i',
  'n',
  'a',
  'l'],
 '-2:bigrams': ['th', 'he'],
 '-2:postag': 'DT',
 '-2:postag[:2]': 'DT',
 '-2:trigrams': ['the'],
 '-2:unigrams': ['t', 'h', 'e'],
 'all_uppercase_letters': 0,
 'caps_mix': 1,
 'contains_capital_letter': 1,
 'contains_drug_prefix': 0,
 'contains_drug_sufix': 0,
 'contain

In [11]:
train_set[598]

[('Drugs', 'NNS', 'O'),
 ('that', 'WDT', 'O'),
 ('raise', 'VBP', 'O'),
 ('the', 'DT', 'O'),
 ('gastrointestinal', 'JJ', 'O'),
 ('pH', 'NN', 'O'),
 ('may', 'MD', 'O'),
 ('lead', 'VB', 'O'),
 ('to', 'TO', 'O'),
 ('an', 'DT', 'O'),
 ('earlier', 'JJR', 'O'),
 ('release', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('duloxetine', 'NN', 'B-drug'),
 ('.', '.', 'O')]

In [12]:
train_texts_entities[397]

('Other strong selective CYP3A4 inhibitors such as ketoconazole can also be expected to increase the exposure of zaleplon.',
 [('ketoconazole', 'drug'), ('zaleplon', 'drug')])

In [13]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=1.33,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.5),
}

# use the same metric for evaluation

labels = list(set(sum(y_train,[])))
labels.remove('O')

f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

rs = RandomizedSearchCV(crf, params_space,
                        cv=4,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=15,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)


Fitting 4 folds for each of 15 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 30.9min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 40.1min finished


RandomizedSearchCV(cv=4, error_score='raise',
          estimator=CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=1.33, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_...e,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False),
          fit_params={}, iid=True, n_iter=15, n_jobs=-1,
          param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x14b5573c8>, 'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x14b5575f8>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          scoring=make_scorer(flat_f1_score, average=weighted, labels=['B-brand', 'B-drug_n', 'B-drug', 'B-group', 'I']),
          verbose=1)

####  Predictions

In [14]:
model = rs.best_estimator_
# model = crf
model.fit(X_train,y_train)
pred_tags = model.predict(X_test)

In [15]:
print(model)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None,
  c1=0.2894710757799365, c2=0.21306562235894833,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)


####  Evaluation

####  Evaluation with scickit learn

In [16]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, pred_tags, labels=sorted_labels, digits=3
))

             precision    recall  f1-score   support

          I      0.750     0.652     0.698       161
    B-brand      0.470     0.915     0.621        59
     B-drug      0.776     0.861     0.816       338
   B-drug_n      0.387     0.108     0.169       111
    B-group      0.705     0.682     0.694       151

avg / total      0.683     0.689     0.669       820



In [17]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(model.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(model.transition_features_).most_common()[-20:])

Top likely transitions:
B-group -> I       0.852876
B-drug_n -> I       0.798307
O      -> O       0.583342
I      -> I       0.308902
O      -> B-group 0.000338
B-drug -> O       -0.000034
O      -> B-brand -0.020661
O      -> B-drug_n -0.040056
B-brand -> O       -0.061517
B-brand -> B-group -0.161544
B-drug -> I       -0.164472
B-brand -> B-brand -0.170059
O      -> B-drug  -0.244170
B-drug_n -> B-group -0.296133
I      -> B-drug  -0.345090
B-drug -> B-drug_n -0.372243
B-group -> B-drug_n -0.377804
B-drug -> B-group -0.455761
B-drug_n -> O       -0.514890
B-drug -> B-brand -0.553121

Top unlikely transitions:
B-drug_n -> B-group -0.296133
I      -> B-drug  -0.345090
B-drug -> B-drug_n -0.372243
B-group -> B-drug_n -0.377804
B-drug -> B-group -0.455761
B-drug_n -> O       -0.514890
B-drug -> B-brand -0.553121
B-group -> B-brand -0.604306
B-drug_n -> B-drug  -0.679047
B-brand -> I       -0.685389
I      -> B-drug_n -0.688441
I      -> B-group -0.755463
B-group -> B-drug  -1.190678
I  

In [18]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(model.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(model.state_features_).most_common()[-30:])

Top positive:
5.311037 B-brand  word[-5:]:pirin
4.223503 B-group  word[-2:]:Is
4.052828 B-group  word[-2:]:Ds
3.559414 B-drug_n word[-5:]:gaine
3.454864 B-drug   word[-5:]:drugs
3.318023 B-brand  word[-2:]:EX
3.158843 B-drug_n word[-2:]:MC
3.117001 B-group  word[-5:]:ylate
2.955492 B-group  word[-5:]:ogens
2.825016 O        +1:bigrams:of
2.726492 O        BOS
2.683978 B-brand  word[-2:]:IL
2.674453 B-drug   word[-5:]:goxin
2.656646 B-drug   isInDB
2.604003 B-brand  word[-2:]:IN
2.559511 B-drug_n word[-5:]:atrol
2.548581 B-brand  word[-2:]:OL
2.515327 B-group  word[-5:]:tacid
2.498567 B-group  word[-5:]:piate
2.471002 B-drug   word[-2:]:ib
2.446081 O        word[-2:]:ay
2.426628 B-drug   word[-5:]:ampin
2.426628 B-drug   word[-4:]:mpin
2.367205 B-brand  word[-5:]:utane
2.357267 B-drug   word[-3:]:cin
2.344016 B-brand  word[-2:]:AN
2.320498 B-group  word[-5:]:rogen
2.313360 O        word[-4:]:tion
2.311073 B-group  word[-5:]:ators
2.290426 B-drug_n word[-5:]:toxin

Top negative:
-1.68550

In [19]:
print('Total execution time: ',(time.time() - first_init)/60, ' minutes')

Total execution time:  44.01821411450704  minutes


In [20]:

'''
## Log of results
date, precision, recall, F1, features, test
14-May, 46.2, 52.1, 48.99, Token length; Prefixes/Suffixes; POS tag; Binary features (+-2); Token position; DrugBank DB; Shape, yes
'''

'\n## Log of results\ndate, precision, recall, F1, features, test\n14-May, 46.2, 52.1, 48.99, Token length; Prefixes/Suffixes; POS tag; Binary features (+-2); Token position; DrugBank DB; Shape, yes\n'